In [47]:
import pandas as pd
import re
import sys

In [48]:
#Modulo que  agrega el path de los modulos al sys.path
from pathlib import Path

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()

# Agregar la ruta al sys.path
if str(ruta_modulos) not in sys.path:
    sys.path.append(str(ruta_modulos))          

# Obtener la ruta al directorio "modulos"
ruta_modulos = Path("../modulos").resolve()
print(f"Ruta de modulos: {ruta_modulos}")  # Verifica la ruta que se obtiene


Ruta de modulos: C:\Users\OPERADOR\Desktop\PROYECTO DIGITALIZACION\digitalizacion_distriapp\backend\src\modulos


In [52]:
# Carga de datos
df_proveedor = cargar_datos("prueba2.csv")
df_productos = cargar_datos("p_STProPrv.csv")

In [50]:
#Copias del DataFrame 
copy_dfProductos = df_productos
copy_dfProveedores = df_proveedor

In [53]:
copy_dfProductos["des_prod"]

0                                FADA MESNA 200 mg a.x 10
1                                    BOTOX 100 U vial x 1
2                                    BOTOX 200 U vial x 1
3                             ENSURE vainilla pvo.x 400 g
4                     ENSURE ADVANCE vainilla pvo.x 400 g
                              ...                        
5618           MASCARA BUCO-NASAL YF02.T:S  - MARCA YUWEL
5619                            LIDOXA jalea x 20 x 25 ml
5620    AGALSIDASA ALFA 1MG/ML F.A. X3.5ML - REPLAGAL ...
5621                            JEVITY PLUS bot.x 1000 ml
5622                CAL SODADA DRAGER DRAGERSORB 800 PLUS
Name: des_prod, Length: 5623, dtype: object

In [44]:
copy_dfProveedores["DESCRIPCIÓN"]

0     HUMIRA AC x 2 (20mg)\nHUMIRA AC x 2 (40mg) HUM...
1                  RINVOQ 15MG RINVOQ 30MG\nRINVOQ 45MG
2                              SKYRIZI SKYRIZI\nSKYRIZI
3     KALETRA SOLUCIÓN\nKALETRA COMPRIMIDOS KALETRA ...
4                                             RITONAVIR
5                                               MAVIRET
6              VENCLEXTA VENCLEXTA VENCLEXTA\nVENCLEXTA
7                         SEVORANE\nSEVORANE QUICK FILL
8                                    SURVANTA\nSURVANTA
9                           ZEMPLAR VIAL\nZEMPLAR 2 MCG
10                                         BOTOX\nBOTOX
11                                              OZURDEX
Name: DESCRIPCIÓN, dtype: object

In [61]:
import re

def clear_column(column):
    """
    Limpiar la columna de descripciones para eliminar términos irrelevantes y normalizar el texto.
    """
    # Expresión regular optimizada para eliminar números, unidades, abreviaciones, y términos irrelevantes
    columna_limpia = re.sub(r'\b(\d+|mg|comp\.?|blisters?|estuche|x|ml|unidades|marca|botella|caja|referencia|cm|g|m|f\.a\.x|jbe\.|sol\.|env\.|lata|vial|pvo\.|sobres|gotero|cápsulas|frascos?|blister|bot\.|mcg|sp|p\.b\.)\b', '', column, flags=re.IGNORECASE)
    
    # Eliminar caracteres especiales como los dos puntos, puntos finales, guiones, comas, etc.
    columna_limpia = re.sub(r'[:\.\-,;()]', '', columna_limpia)

    # Eliminar múltiples espacios en blanco y normalizar la columna
    columna_limpia = re.sub(r'\s+', ' ', columna_limpia).strip()

    # Convertir todo el texto a minúsculas
    columna_limpia = columna_limpia.lower()

    # Eliminar posibles palabras vacías adicionales que puedan quedar
    columna_limpia = re.sub(r'\b(?:de|la|el|en|a|y|con|para|del|sobre|en|por|para|con|medicamento|tratamiento|ac)\b', '', columna_limpia).strip()

    # Opción adicional para quitar palabras repetidas o redundantes (como 'skyrizi skyrizi')
    columna_limpia = re.sub(r'\b(\w+)\s+\1\b', r'\1', columna_limpia)

    # Eliminar términos relacionados con la variabilidad en dosis o presentación (por ejemplo, "20mg", "30mg", etc.)
    columna_limpia = re.sub(r'\b(\d+mg|\d+g|\d+ml)\b', '', columna_limpia)

    return columna_limpia


# Función para obtener coincidencias entre las columnas y agregar una columna 'match'
def obtener_coincidencias_con_match(productos_df, proveedores_df, col_productos, col_proveedores):
    """
    Realiza un match entre las columnas especificadas de dos DataFrames y agrega una columna 'match'.
    
    Parámetros:
        productos_df (DataFrame): DataFrame de productos.
        proveedores_df (DataFrame): DataFrame de proveedores.
        col_productos (str): Nombre de la columna en productos.
        col_proveedores (str): Nombre de la columna en proveedores.
        
    Retorna:
        DataFrame: DataFrame con las coincidencias y una columna 'match'.
    """
    # Limpiar ambas columnas antes de realizar la comparación
    productos_df[col_productos] = productos_df[col_productos].apply(lambda x: clear_column(str(x)))
    proveedores_df[col_proveedores] = proveedores_df[col_proveedores].apply(lambda x: clear_column(str(x)))
    
    resultados = []

    # Iterar por cada descripción en proveedores
    for _, proveedor in proveedores_df.iterrows():
        descripcion_proveedor = proveedor[col_proveedores]
        
        # Buscar coincidencias en los productos
        coincidencias = productos_df[
            productos_df[col_productos].str.contains(descripcion_proveedor, case=False, na=False)
        ]
        
        # Agregar coincidencias al resultado
        if not coincidencias.empty:
            for _, producto in coincidencias.iterrows():
                resultado = {**proveedor.to_dict(), **producto.to_dict(), "match": True}
                resultados.append(resultado)
        else:
            # Si no hay coincidencias, agregar el proveedor con valores nulos para los productos
            resultado = proveedor.to_dict()
            for col in productos_df.columns:
                resultado[col] = None
            resultado["match"] = False
            resultados.append(resultado)
    
    return pd.DataFrame(resultados)

# Llamar Match a la función con tus DataFrames
resultado_df = obtener_coincidencias_con_match(
    productos_df=copy_dfProductos,
    proveedores_df=copy_dfProveedores,
    col_productos="des_prod",
    col_proveedores="DESCRIPCIÓN"
)


In [62]:
# Mostrar el DataFrame resultante
resultado_df

0                                        humira  humira
1                                         rinvoq rinvoq
2                                               skyrizi
3     kaletra solución kaletra comprimidos kaletra c...
4                                             ritonavir
5                                               maviret
6                                             venclexta
7                                             venclexta
8                                   sevorane quick fill
9                                   sevorane quick fill
10                                             survanta
11                                              zemplar
12                                                botox
13                                                botox
14                                                botox
15                                                botox
16                                                botox
17                                              